In [2]:
import matplotlib.pypp as plt
import numpy as np

In [5]:
data = np.transpose(np.loadtxt("peakpositioner.txt",skiprows=1))
thetas = data[0]
peakpos = data[1]

In [6]:
plt.plot(thetas,peakpos)

AttributeError: module 'matplotlib' has no attribute 'plot'